In [160]:
import wandb
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: taoprajjwal (pj-runs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [179]:
import torch
from tqdm import tqdm
import math

def generate_low_inner_product_vectors(n, epsilon, device='cpu', seed=None):
    if seed is not None:
        torch.manual_seed(seed)

    # Compute k = exp(epsilon^2 * n / 4)
    k = int(math.exp((epsilon**2 * n) / 4))
    
    # Generate k x n random ±1 vectors
    signs = torch.randint(0, 2, (k, n), device=device, dtype=torch.float32) * 2 - 1  # ±1
    vectors = signs / math.sqrt(n)  # Normalize each to have norm 1

    # Compute inner products
    ip_matrix = torch.matmul(vectors, vectors.T)
    mask = torch.eye(k, device=device).bool()
    ip_matrix[mask] = 0  # zero out diagonal

    max_ip = ip_matrix.abs().max().item()
    success = (ip_matrix.abs() < epsilon).all().item()

    print(f"Generated {k} vectors in {n} dimensions.")
    print(f"Maximum off-diagonal inner product: {max_ip:.4f}")
    print("All inner products < ε:", success)

    return vectors


In [147]:
vecs = generate_low_inner_product_vectors(1000, 0.2)

Generated 22026 vectors in 1000 dimensions.
Maximum off-diagonal inner product: 0.1900
All inner products < ε: True


In [148]:
torch.linalg.norm(vecs, dim=1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000])

In [149]:
proj = torch.randn(1000, 500)

In [150]:
stud_vec_2 = torch.randn(22026, 500)
stud_vec_2 = stud_vec_2 / stud_vec_2.norm(dim=1, keepdim=True)

In [151]:
stud_vec_2 = stud_vec_2.requires_grad_(True)

In [152]:
stud_vecs = vecs @ proj
stud_vecs = stud_vecs / stud_vecs.norm(dim=1, keepdim=True)

In [153]:
student_vecs = stud_vecs.requires_grad_(True)

In [178]:
def luby_mis_from_dense(adj_dense, max_iters=1000, device='cpu'):
    """
    Compute an approximate maximal independent set using Luby's algorithm.
    
    Args:
        adj_dense (torch.Tensor): Dense n x n adjacency matrix with 1s as edges, 0s elsewhere.
        max_iters (int): Maximum iterations to try.
        device (str): 'cpu' or 'cuda'.
    
    Returns:
        torch.Tensor: Indices of nodes in the MIS.
    """
    # Convert to sparse format for efficiency
    adj_dense = adj_dense.to(device)
    n = adj_dense.size(0)
    adj_sparse = adj_dense.to_sparse().coalesce()

    in_set = torch.zeros(n, dtype=torch.bool, device=device)
    remaining = torch.ones(n, dtype=torch.bool, device=device)

    for _ in range(max_iters):
        if not remaining.any():
            break

        priorities = torch.rand(n, device=device)
        priorities[~remaining] = -1e9  # Effectively ignore these

        # Compute max neighbor priority
        neighbor_max = torch.sparse.mm(adj_sparse, priorities.unsqueeze(1)).squeeze(1)

        selected = (priorities > neighbor_max) & remaining
        in_set[selected] = True

        # Mark neighbors of selected for removal
        selected_mask = selected.to(adj_dense.dtype).unsqueeze(0)  # (1 x n)
        neighbors_of_selected = (selected_mask @ adj_dense).squeeze(0).bool()

        to_remove = selected | neighbors_of_selected
        remaining[to_remove] = False

    return torch.nonzero(in_set, as_tuple=False).squeeze(1)

In [183]:
def get_adjc(vec, eps):
    norm_vec = vec / vec.norm(dim=1, keepdims=True)
    ip_matrix = vec@vec.T
    mask = torch.eye(len(vec)).bool()
    ip_matrix[mask] = 0

    return (ip_matrix.abs() < eps).float()

In [143]:
s= luby_mis_from_dense(1-success,100)

no remains


In [128]:
len(s)

20627

In [101]:
len(s)

20624

In [139]:
selected_m = ip_matrix[s][:, s]

In [144]:
selected_m.shape

torch.Size([20621, 20621])

In [145]:
((selected_m.abs() < 0.2).float().sum(dim = 1) == 20618).unique(return_counts=True)

(tensor([False]), tensor([20621]))

In [120]:
(selected_m.abs()< 0.2).float().sum(dim=1).argmin()

tensor(12284)

In [123]:
selected_m[12284][11209]

tensor(0.2004, grad_fn=<SelectBackward0>)

In [59]:
adj = generate_sparse_torch_adj(22000, 0.05)

In [62]:
adj

tensor(indices=tensor([[    0,     0,     0,  ..., 21999, 21999, 21999],
                       [    6,    15,    27,  ..., 21984, 21985, 21996]]),
       values=tensor([2., 1., 2.,  ..., 1., 1., 1.]),
       size=(22000, 22000), nnz=47176416, layout=torch.sparse_coo)

In [78]:
ip_matrix = stud_vec_2@ stud_vec_2.T
mask = torch.eye(len(stud_vec_2)).bool()
ip_matrix[mask] = 0  # zero out diagonal


In [ ]:
n = greedy_max_independent(ip_matrix,0.25)

In [43]:
success = (ip_matrix.abs() < 0.2)
i = torch.argmin((success * (1- ip_matrix.abs())).sum(dim=1))

In [125]:
success = (ip_matrix.abs() < 0.2).float()

In [18]:
success.all().item()

False

In [39]:
22026*22026 - success.sum()

tensor(3232.)

In [162]:
from similarity_measures import LinearMeasure, CKA
import similarity

In [170]:
lin = LinearMeasure(approx=False)
cka = CKA(biased=False)
measure = similarity.make("measure/netrep/procrustes-distance=euclidean")

In [171]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(vecs, stud_vec_2)
dl = DataLoader(dataset, batch_size=256, shuffle=False)


In [182]:
optim = torch.optim.AdamW([stud_vec_2])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="cka_minim")
for epoch in range(10):
    for batch in tqdm(dl, desc=f"Epoch {epoch+1}"):
        a,b = batch
        loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = measure(b.detach().numpy(), a.detach().numpy())
        adj = get_adjc(stud_vec_2, 0.2)
        s = luby_mis_from_dense(1-adj)
        wandb.log( {"shape": shape, "cka":loss.item(), "eps-orth-vectors": len(s)})
        loss.backward()
        optim.step()
        optim.zero_grad()
wandb.finish()

Epoch 10: 100%|██████████| 87/87 [03:50<00:00,  2.65s/it]


cka,██▆▆▅▅▃▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps-orth-vectors,███▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▂▂▂▂▁▁▁
shape,▅▅▅▅▅▅▆▆▄▇▃▇▃▃▇▇▂▇▂█▇▁▇▁▇▇▁▇▁▇█▁█▂███▁█▂
cka,0.03313
eps-orth-vectors,13236
shape,0.59853


In [175]:
s = luby_mis_from_dense(1-adj)

no remains


In [33]:
stud_vec_after = stud_vec_2 / stud_vec_2.norm(dim=1, keepdim=True)

In [34]:
ip_matrix = torch.matmul(stud_vec_after, stud_vec_after.T)
mask = torch.eye(22026).bool()
ip_matrix[mask] = 0  # zero out diagonal

max_ip = ip_matrix.abs().max().item()
success = (ip_matrix.abs() < 0.2).float()

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [35]:
22026*22026 - success.sum()

tensor(7328.)

In [36]:
7328/2

3664.0

In [ ]:
optim = torch.optim.AdamW([stud_vec_2])
wandb.init(project="synthetic_minimization", entity="pj-runs", name="cka_minim")
for epoch in range(20):
    for batch in dl:
        a,b = batch
        loss = 1- (cka(b.unsqueeze(1), a.unsqueeze(1)))
        shape = measure(b.detach().numpy(), a.detach().numpy())
        wandb.log( {"shape": shape, "cka":loss.item()})
        loss.backward()
        optim.step()
        optim.zero_grad()

In [164]:
loss

tensor(0.2200, grad_fn=<RsubBackward1>)

In [19]:
lin(b.unsqueeze(1), a.unsqueeze(1))

tensor(0., grad_fn=<MeanBackward0>)

In [23]:
measure(b.detach().numpy(), a.detach().numpy())

3.4752114467949236